# Steam Game Review Scraper

Scrape game review data from Steam, including the user, profile link, and the content of the review itself

In [6]:

from selenium.webdriver import ChromeOptions, Chrome
from selenium.webdriver.common.keys import Keys
import re
from time import sleep
from datetime import datetime
from openpyxl import Workbook
import csv

## Requirements
You'll need to install the following libraries before beginning this project:
- [Selenium](https://www.selenium.dev/downloads/) : for automating the web browser; this can be involved... so check my [short YouTube video](https://youtu.be/9XAH_TvxwLg) for a walkthrough.
- [OpenPyXL](https://openpyxl.readthedocs.io/en/stable/) : for saving the data to an Excel spreadsheet (optional)

## Example
If you want to see an example of the output... you can see the results of me running the scaper for about 5 minutes on a particular game  
[Click to view Excel file](https://drive.google.com/file/d/1Ld04lwFY7OjIMU2wJxRcgdPvJ0o43BRo/view?usp=sharing)

## Getting started

Lookup the game id by doing a search on steam, navigate to the game homepage, and then get the number embedded in the URL before the game title.

In [7]:
# star wars: squadrons (1222730)
# scrap mechanic (387990)
game_id = 1222730

The url template below can be altered to filter by sentiment, language, and recency.  

Check the [website](https://steamcommunity.com/app/387990/positivereviews/?browsefilter=mostrecent) to see what options are available. For this project, I'm going to focus on **Positive** reviews only and sort by **Most Recent**.

In [8]:
template = 'https://steamcommunity.com/app/{}/positivereviews/?browsefilter=mostrecent'
template_with_language = 'https://steamcommunity.com/app/{}/positivereviews/?browsefilter=mostrecent&filterLanguage=russian'

url = template_with_language.format(game_id)

In [9]:
# setup driver
options = ChromeOptions()
# options.use_chromium = True
driver = Chrome(options=options)

Maximize the window and get the starting url

In [10]:
driver.maximize_window()
driver.get(url)

## Scrape the data

The page is continously scrolling, so you'll need to grab the cards, then scroll down to the bottom and repeat until finished. For this project, we are going to collect the following information:
- Steam ID
- Profile URL
- Review Text
- Review Recommendation
- Review Length (chars)
- Play Hours
- Date Posted

In [11]:
# get current position of y scrollbar
last_position = driver.execute_script("return window.pageYOffset;")

reviews = []
review_ids = set()
running = True

while running:
    # get cards on the page
    cards = driver.find_elements_by_class_name('apphub_Card')

    for card in cards[-20:]:  # only the tail end are new cards

        # gamer profile url
        profile_url = card.find_element_by_xpath('.//div[@class="apphub_friend_block"]/div/a[2]').get_attribute('href')

        # steam id
        steam_id = profile_url.split('/')[-2]
        
        # check to see if I've already collected this review
        if steam_id in review_ids:
            continue
        else:
            review_ids.add(steam_id)

        # username
        user_name = card.find_element_by_xpath('.//div[@class="apphub_friend_block"]/div/a[2]').text

        # language of the review
        date_posted = card.find_element_by_xpath('.//div[@class="apphub_CardTextContent"]/div').text
        review_content = card.find_element_by_xpath('.//div[@class="apphub_CardTextContent"]').text.replace(date_posted,'').strip()    

        # review length
        review_length = len(review_content.replace(' ', ''))    

        # recommendation
        thumb_text = card.find_element_by_xpath('.//div[@class="reviewInfo"]/div[2]').text
        thumb_text    

        # amount of play hours
        play_hours = card.find_element_by_xpath('.//div[@class="reviewInfo"]/div[3]').text
        play_hours    

        # save review
        review = (steam_id, profile_url, review_content, thumb_text, review_length, play_hours, date_posted)
        reviews.append(review)    
        
    # attempt to scroll down thrice.. then break
    scroll_attempt = 0
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")    
        sleep(0.5)
        curr_position = driver.execute_script("return window.pageYOffset;")
        
        if curr_position == last_position:
            scroll_attempt += 1
            sleep(0.5)
            
            if curr_position >= 3:
                running = False
                break
        else:
            last_position = curr_position
            break  # continue scraping the results

# shutdown the web driver
driver.close()

AttributeError: 'WebDriver' object has no attribute 'find_elements_by_class_name'

## Save the results

You can push the data wherever you want. However, for this project, I'm going to save the data to an Excel spreadsheet using the [OpenPyXL](https://openpyxl.readthedocs.io/en/stable/) library

In [ ]:
# save the file to Excel Worksheet
wb = Workbook()
ws = wb.worksheets[0]
ws.append(['SteamId', 'ProfileURL', 'ReviewText', 'Review', 'ReviewLength(Chars)', 'PlayHours', 'DatePosted'])
for row in reviews:
    ws.append(row)
    
today = datetime.today().strftime('%Y%m%d')    
wb.save(f'Steam_Reviews_{game_id}_{today}.xlsx')    
wb.close()

In [ ]:
# save the file to a CSV file
today = datetime.today().strftime('%Y%m%d')   
with open(f'Steam_Reviews_{game_id}_{today}.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['SteamId', 'ProfileURL', 'ReviewText', 'Review', 'ReviewLength(Chars)', 'PlayHours', 'DatePosted'])
    writer.writerows(reviews)